<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Keras-AutoEncoder" data-toc-modified-id="Keras-AutoEncoder-1">Keras AutoEncoder</a></span><ul class="toc-item"><li><span><a href="#MLP---PLC" data-toc-modified-id="MLP---PLC-1.1">MLP - PLC</a></span></li><li><span><a href="#Autoencoder-on-Iris-Dataset" data-toc-modified-id="Autoencoder-on-Iris-Dataset-1.2">Autoencoder on Iris Dataset</a></span></li></ul></li></ul></div>

# Keras AutoEncoder

## MLP - PLC

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop

In [ ]:
batch_size = 50
num_classes = 3
epochs = 200

In [ ]:
import pandas as pd

plc=pd.read_csv("plc2.csv")
plc.drop(["Date", "Time"], axis=1, inplace=True)

# Shuffling
plc=plc.sample(frac=1).reset_index(drop=True)
plc_train=plc.iloc[0:100,:]
plc_test=plc.iloc[100:150,:]

In [ ]:
plc_train.head()

In [ ]:
plc_test.head()

In [ ]:
# source, target
x_train=plc_train.iloc[:,0:6].values
x_test=plc_test.iloc[:,0:6].values
y_train=plc_train.iloc[:,6:]
y_test=plc_test.iloc[:,6:]

In [ ]:
x_train[:5]

In [ ]:
y_train[:5]

In [ ]:
# encoder={k:v for v,k in enumerate(y_train.drop_duplicates())}
# encoder
#sets=np.unique(plc.iloc[:,6:].values)
sets=plc.iloc[:,6:].drop_duplicates()["state"].tolist()
sets

In [ ]:
encoder={k:v for v,k in enumerate(sets)}

y_train=[ encoder[i] for i in y_train["state"].tolist() ]
y_train = keras.utils.to_categorical(y_train, num_classes)

y_test=[ encoder[i] for i in y_test["state"].tolist() ]
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# 데이터 shape 을 확인해 봅니다.
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

In [ ]:
model = Sequential()
model.add(Dense(30, activation='relu', input_shape=(6,)))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## Autoencoder on Iris Dataset

* Source : https://www.kaggle.com/shivam1600/autoencoder-on-iris-dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Importing Data
data = pd.read_csv("iris.csv")
display(data.head())

x_train, x_test, y_train, y_test = train_test_split(data[['Sepal.Length', 'Sepal.Width',
                                                          'Petal.Length', 'Petal.Width']],
                                                    data['Species'],test_size=0.1, random_state=1)

In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import RMSprop

# this is the size of our encoded representations
encoding_dim1 = 6
encoding_dim2 = 4
encoding_dim3 = 2
input_dim = 4

# this is our input placeholder
input_img = Input(shape=(input_dim,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim1)(input_img)
encoded = Dense(encoding_dim2)(encoded)
encoded = Dense(encoding_dim3)(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(encoding_dim2)(encoded)
decoded = Dense(encoding_dim1)(decoded)
decoded = Dense(input_dim)(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)


# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)
#encoder = decoder_layer = autoencoder.layers[3]

# create a placeholder for an encoded (2-dimensional) input
encoded_input = Input(shape=(encoding_dim3,))
# retrieve the last layer of the autoencoder model

decoder_layer = autoencoder.layers[-3](encoded_input)
decoder_layer = autoencoder.layers[-2](decoder_layer)
decoder_layer = autoencoder.layers[-1](decoder_layer)

# create the decoder model
decoder = Model(encoded_input, decoder_layer)
opt=RMSprop(lr=0.001)
autoencoder.compile(loss='mean_squared_error', optimizer=opt)
autoencoder.summary()

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=333,
                batch_size=123,
                shuffle=True,
                validation_data=(x_test, x_test),
               callbacks=[])

# encode and decode some data points
# note that we take them from the *test* set
encoded_datapoints = encoder.predict(x_test)
decoded_datapoints = decoder.predict(encoded_datapoints)

print('Original Datapoints :')
print(x_test)
print('Reconstructed Datapoints :')
print(decoded_datapoints)

In [ ]:
# Plotting Encoded Features
encoded_dataset = encoder.predict(x_test[['Sepal.Length', 'Sepal.Width',
                                        'Petal.Length', 'Petal.Width']])

In [ ]:
# Encode Dataset
plt.scatter(encoded_dataset[:,0], encoded_dataset[:,1], c=y_test.astype('category').cat.codes)
plt.show()

In [ ]:
# Decoded Dataset(reconstruncted)
decoded_dataset=decoder.predict(encoded_dataset)
plt.scatter(decoded_dataset[:,2], decoded_dataset[:,3], c=y_test.astype('category').cat.codes)
plt.show()

In [ ]:
# Orignal Data
plt.scatter(x_test['Petal.Length'], x_test['Petal.Width'],c=y_test.astype('category').cat.codes)
plt.show()